In [0]:
import pandas as pd
import numpy as np
!pip install contractions
import contractions
import re
import nltk
import spacy
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')



In [0]:
# import a file from drive : 
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1g2BefhpdVbg6aeFcngD52p_HLza6nQDm'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('IMDB Dataset.csv')  
imdb = pd.read_csv('IMDB Dataset.csv')


In [0]:
x = imdb["review"]
y = [1 if word=="positive" else 0 for word in imdb["sentiment"]]

In [0]:
def preprocess_text(text):
  #standarize text to lowercase
  text = text.lower()

  #remove html tags
  remove_tags = re.compile('<.*?>')
  text= re.sub(remove_tags, '', text)

  #remove special characters and numbers
  remove_special_char = re.compile('[^a-z ]', re.IGNORECASE)
  text = remove_special_char.sub('', text)

  #expand decontractions
  text = contractions.fix(text)

  #setting stopwords
  stop_words = set(stopwords.words('english')) 

  #for every word in the review, if it's not a stopword, replace it with it's lemma, otherwise go for next word
  processed_text = []
  text = text.split()
  lemmatizer = WordNetLemmatizer()

  for word in text:
    if word in stop_words:
      continue
      
    word = lemmatizer.lemmatize(word)
    word = lemmatizer.lemmatize(word, 'v')
    processed_text.append(word)

  text = ' '.join(processed_text)

  return text

In [0]:

x = []

for review in list(imdb['review']):
  clean = preprocess_text(review)
  x.append(clean)
  if (x.index(clean) % 1000 == 0):
    print(x.index(clean))


In [40]:
from keras.preprocessing import sequence 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

y = [1 if sentiment=='positive' else 0 for sentiment in list(imdb['sentiment'])]
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(x)
x = pad_sequences(tokenizer.texts_to_sequences(x), maxlen=100)
max_words = 100
vocabulary_size = 50000

# Build the model
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 32)           1600000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,653,301
Trainable params: 1,653,301
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [0]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [44]:
batchSize = 64
epochs = 3
hist = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epochs, verbose=1, shuffle=True, validation_split=0.1)






Train on 36000 samples, validate on 4000 samples
Epoch 1/3





36000/36000 [==============================] - 95s 3ms/step - loss: 0.3625 - acc: 0.8321 - val_loss: 0.3097 - val_acc: 0.8770
Epoch 2/3
36000/36000 [==============================] - 94s 3ms/step - loss: 0.1968 - acc: 0.9269 - val_loss: 0.3020 - val_acc: 0.8748
Epoch 3/3
36000/36000 [==============================] - 95s 3ms/step - loss: 0.1205 - acc: 0.9567 - val_loss: 0.3698 - val_acc: 0.8690


In [45]:
scores = model.evaluate(xTest, yTest, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8613


In [53]:
predicted = ["I advise you not to repeat this again"]
predicted_tokenized = pad_sequences(tokenizer.texts_to_sequences(predicted), maxlen=100)
model.predict_classes(predicted_tokenized)

array([[0]], dtype=int32)